In [1]:
from scipy.optimize import minimize_scalar

from UnifiedMomentumModel.Momentum import UnifiedMomentum

# The new Betz Limit

In [2]:
model = UnifiedMomentum()


def to_minimize(x):
    Ctprime = x
    sol = model(Ctprime, 0)
    return -sol.Cp


sol = minimize_scalar(to_minimize, bounds=(0.01, 3))
Ctprime_maximising = sol.x
sol = model(Ctprime_maximising, 0)
print(f"Maximum $C_p$ = {sol.Cp[0]:.4f} ({(sol.Cp[0]/(16/27) - 1) * 100:.4f}% higher)")
print(f"Maximising $a_n$ = {sol.an[0]:.4f} ({(sol.an[0]/(1/3) - 1) * 100:.4f}% higher)")
print(
    f"Maximising $C_T'$ = {sol.Ctprime[0]:.4f} ({(sol.Ctprime[0]/2 - 1) * 100:.4f}% higher)"
)

Maximum $C_p$ = 0.5984 (0.9792% higher)
Maximising $a_n$ = 0.3451 (3.5339% higher)
Maximising $C_T'$ = 2.1305 (6.5269% higher)


# Model accuracy vs LES

In [3]:
from pathlib import Path
from typing import Dict, Tuple
from dataclasses import dataclass
from scipy.interpolate import UnivariateSpline
import os

import numpy as np
from numpy.typing import ArrayLike

from scipy.io import loadmat


lesdir = Path(os.getcwd()) / "LES_data"

# Collect data to compare
Data to compared is collected in a dictionary of lists of `MomentumSolution` objects (`Dict[str, List[MomentumSolution]]`)

In [4]:
@dataclass
class ContourData:
    """
    Stores the Cp grid to be plotted in a contour plot.
    """

    yaw: ArrayLike  # 1D [deg]
    Ctprime: ArrayLike  # 1D
    Cp: ArrayLike  # 2D

    def data(self) -> Tuple[ArrayLike]:
        return self.yaw, self.Ctprime, self.Cp

    def power_maximising_setpoints(self) -> (ArrayLike, ArrayLike):
        return self.yaw, self.Ctprime[np.argmax(self.Cp, axis=1)]


# Set up results dictionary
results: Dict[str, ContourData] = {}

In [5]:
data = loadmat(lesdir / "Cp_les_contour_data.mat")

Ctprimes = data["ctp_les"][0]
yaws = data["yaw_les"][0]
Cps = data["cp_les"]

results["LES"] = ContourData(yaws, Ctprimes, Cps)

# Generate `classical` model data

In [6]:
an = Ctprimes / (Ctprimes + 4)  # From C_T = 4a(1-a) and C_t = C_T' (1-a)^2


Ctprime_mesh, yaw_mesh = np.meshgrid(Ctprimes, yaws)
Cps = Ctprime_mesh * (1 - an) ** 3 * np.cos(np.deg2rad(yaw_mesh)) ** 3

results["classical"] = ContourData(yaws, Ctprimes, Cps)

# Generate `unified` momentum model Data

In [7]:
Cps = np.zeros((len(yaws), len(Ctprimes)))
model = UnifiedMomentum()

for i, yaw in enumerate(yaws):
    for j, Ctprime in enumerate(Ctprimes):
        sol = model(Ctprime, np.deg2rad(yaw))
        Cps[i, j] = 0 if np.isnan(sol.Cp) else sol.Cp

results["unified"] = ContourData(yaws, Ctprimes, Cps)

/home/jaime/Repositories/Unified-Momentum-Model/UnifiedMomentumModel/Momentum.py:159: RuntimeWarning: divide by zero encountered in divide
  np.cos(yaw)
/home/jaime/Repositories/Unified-Momentum-Model/UnifiedMomentumModel/Momentum.py:169: RuntimeWarning: invalid value encountered in divide
  -dp / (0.5 * Ctprime * np.cos(yaw) ** 2)
/home/jaime/Repositories/Unified-Momentum-Model/UnifiedMomentumModel/Momentum.py:170: RuntimeWarning: invalid value encountered in divide
  + (1 - u4**2 - v4**2) / (Ctprime * np.cos(yaw) ** 2)
/home/jaime/Repositories/Unified-Momentum-Model/UnifiedMomentumModel/Utilities/FixedPointIteration.py:57: RuntimeWarning: All-NaN slice encountered
  max_resid = [np.nanmax(np.abs(_r)) for _r in residuals]


# Generate `unified` momentum model Data *With linear pressure only*

In [10]:
Cps = np.zeros((len(yaws), len(Ctprimes)))
model = UnifiedMomentum(cached=False, max_iter=0)

for i, yaw in enumerate(yaws):
    for j, Ctprime in enumerate(Ctprimes):
        sol = model(Ctprime, np.deg2rad(yaw))

        Cps[i, j] = 0 if np.isnan(sol.Cp) else sol.Cp

results["linear"] = ContourData(yaws, Ctprimes, Cps)

## Error

In [11]:
error_classical = np.mean(np.abs(results["classical"].Cp - results["LES"].Cp))
error_linear = np.mean(np.abs(results["linear"].Cp - results["LES"].Cp))
error_unified = np.mean(np.abs(results["unified"].Cp - results["LES"].Cp))

error_classical, error_linear, error_unified

print(
    f"Unified presents a {(error_unified/error_classical-1)*100:.1f}% error reduction compared to classical."
)
print(
    f"Unified presents a {(error_unified/error_linear-1)*100:.1f}% error reduction compared to linear."
)

Unified presents a -84.3% error reduction compared to classical.
Unified presents a -21.0% error reduction compared to linear.
